In [1]:
import pandas as pd

storageInfo = {
    "Pantry": [
        "Pantry_Min",
        "Pantry_Max",
        "Pantry_Metric",
        "Pantry_tips",
        "DOP_Pantry_Min",
        "DOP_Pantry_Max",
        "DOP_Pantry_Metric",
        "DOP_Pantry_tips",
        "Pantry_After_Opening_Min",
        "Pantry_After_Opening_Max",
        "Pantry_After_Opening_Metric"
    ],
    "Refrigerate": [
        'Refrigerate_Min',
        'Refrigerate_Max',
        'Refrigerate_Metric',
        'Refrigerate_tips',
        'DOP_Refrigerate_Min',
        'DOP_Refrigerate_Max',
        'DOP_Refrigerate_Metric',
        'DOP_Refrigerate_tips',
        'Refrigerate_After_Opening_Min',
        'Refrigerate_After_Opening_Max',
        'Refrigerate_After_Opening_Metric',
        'Refrigerate_After_Thawing_Min',
        'Refrigerate_After_Thawing_Max',
        'Refrigerate_After_Thawing_Metric'
    ],
    "Freeze" : [
        "Freeze_Min",
        "Freeze_Max",
        "Freeze_Metric",
        "Freeze_Tips",
        "DOP_Freeze_Min",
        "DOP_Freeze_Max",
        "DOP_Freeze_Metric",
        "DOP_Freeze_Tips"
    ]
}

df = pd.read_excel('datasets/FoodKeeper-Data.xls', sheet_name = 'Product')

def isNaN(string): return string != string

def keywordFinder(key):
    for index  in range(len(df['Keywords'])-1):
        keywordBank = df['Keywords'].iloc[index]
        if isNaN(keywordBank) == False:
            temp = keywordBank.split(",")
            for keys in temp:
                if keys.lower() == key.lower():
                    print(keywordBank.lower(), index)
                    print("Name: ", df['Name'].iloc[index])

def foodNameFinder(key):
    for index  in range(len(df['Name'])-1):
        nameSearch = df['Name'].iloc[index]
        ifFound = key.lower().capitalize()
        if nameSearch == ifFound: return [index, nameSearch]

def getColumns(info, index):
    temp = []
    if storageInfo.get(info):
        for column in storageInfo[info]:
            if isNaN(df[column].iloc[index]) == False:
                temp.append(column)
    return temp

def entityFound(key):
    return True if foodNameFinder(key) != None else False

def foodStorage(foodName, information):
    tempString = ''
    foundFood = foodNameFinder(foodName)
    if entityFound(foodName):
        index = foundFood[0]
        foodName = foundFood[1]
    else: return ['No Tip to give!']
    info = information.lower().capitalize()
    tempCol = getColumns(info, index)
    all_info = []
    
    for col in tempCol:
        tempString = ''
        if "Metric" in col:
            if "DOP" not in col:
                if isNaN(df[info + "_Metric"].iloc[index]) == False and df[info + "_Metric"].iloc[index] not in ['Days', 'Weeks', 'Months', 'Years']: tempString = f'{info.capitalize()} is {df[info + "_Metric"].iloc[index]}'
            if 'DOP' in col:
                if isNaN(df["DOP_" + info + "_Metric"].iloc[index]) == False and df["DOP_" + info + "_Metric"].iloc[index] not in ['Days', 'Weeks', 'Months', 'Years']: tempString = f'{info.capitalize()} is {df["DOP_" + info + "_Metric"].iloc[index]}'
            if 'After' in col and info == "Refrigerate":
                if isNaN(df[info + "_After_Thawing_Metric"].iloc[index]) == False and df[info + "_After_Thawing_Metric"].iloc[index] not in ['Days', 'Weeks', 'Months', 'Years']: tempString = f'{info.capitalize()} is {df[info + "_After_Thawing_Metric"].iloc[index]}'
                if isNaN(df[info + "_After_Opening_Metric"].iloc[index]) == False and df[info + "_After_Opening_Metric"].iloc[index] not in ['Days', 'Weeks', 'Months', 'Years']: tempString = f'{info.capitalize()} is {df[info + "_After_Opening_Metric"].iloc[index]}'
        if "DOP" in col and "Min" in col:
            min = df['DOP_' + info + '_Min'].iloc[index]
            max = df['DOP_' + info + '_Max'].iloc[index]
            if isNaN(min) == False and isNaN(max) == False:
                if min == max: tempString = f'The Date of Perish for {foodName} is {int(min)} {df["DOP_" + info + "_Metric"].iloc[index]}'
                else: tempString = f'The Date of Perish for {foodName} is from {int(min)} to {int(max)} {df["DOP_" + info + "_Metric"].iloc[index]}'
        if "Min" in col and "DOP" not in col:
            min = df[info + '_Min'].iloc[index]
            max = df[info + '_Max'].iloc[index]
            if isNaN(min) == False and isNaN(max) == False:
                if min == max: tempString = f'{foodName.capitalize()} can be stored in the {info} for {int(min)} {df[info + "_Metric"].iloc[index]}'
                else: tempString = f'{foodName.capitalize()} can be stored in the {info} for {int(min)} to {int(max)} {df[info + "_Metric"].iloc[index]}'
        if "After" in col and "Min" in col  and info == "Refrigerate":
            if "Thawing" in col:
                min = df[info + '_After_Thawing_Min'].iloc[index]
                max = df[info + '_After_Thawing_Max'].iloc[index]
                if isNaN(min) == False and isNaN(max) == False:
                    if min == max: tempString = f'{foodName.capitalize()} can be stored after thawing in the {info} for {int(min)} {df[info + "_After_Thawing_Metric"].iloc[index]}'
                    else: tempString = f'{foodName.capitalize()} can be stored after thawing in the {info} for {int(min)} to {int(max)} {df[info + "_After_Thawing_Metric"].iloc[index]}'
            else:
                min = df[info + '_After_Opening_Min'].iloc[index]
                max = df[info + '_After_Opening_Max'].iloc[index]
                if isNaN(min) == False and isNaN(max) == False:
                    if min == max: tempString = f'{foodName.capitalize()} can be stored after opened in the {info} for {int(min)} {df[info + "_After_Opening_Metric"].iloc[index]}'
                    else: tempString = f'{foodName.capitalize()} can be stored after opened in the {info} for {int(min)} to {int(max)} {df[info + "_After_Opening_Metric"].iloc[index]}'
        if "tips" in col or "Tips" in col:
            if "Freeze" in col:
                if "DOP" in col: tempString += f'A tip about {foodName} the Date of Perish is {df["DOP_" + info + "_Tips"].iloc[index]}'
                else: tempString = f'A tip about {foodName} is that {df[info + "_Tips"].iloc[index]}'
            else:
                if "DOP" in col: tempString += f'A {foodName} tip about the Date of Perish is {df["DOP_" + info + "_tips"].iloc[index]}'
                else: tempString = f'A tip about {foodName} is that {df[info + "_tips"].iloc[index]}'
    
        if tempString != '': all_info.append(tempString)                   
    return all_info if len(all_info) else ['No Tip to give!']


In [2]:
# import random
# import language_tool_python
# tool = language_tool_python.LanguageTool('en-US')

# foundKeywords = ['rice', 'butter', 'salt', 'pepper', 'fish', 'fruit']
# foundKeywords = ['Coconut']
# for k in foundKeywords:
#     print(k)
#     keywordFinder(k)
#     print('\n')

'''
Discoveries...
1. Using the foundKeywords, find semantics of tweet to utilize particular keywords.
    - natural language processing (simpler)
    - "Red Hot Chili Peppers"

Things to fix...
1. Refrigerator, Freezer (ending words)
2. some cases produce nan, figure out why
3. type casting all min and max values to int, not completly working (possible fix after above)
4. Test all index for accuracy (is it really not tip to give?)
'''

# types = ['Pantry', 'Refrigerate', 'Freeze']

# numIndex = 3
# for i in range(numIndex):
#     index = random.randint(0, 651)
#     print("=== ", df["Name"].iloc[index], " ===", f' ({index + 2})')
#     for t in types:
#         all_tips = foodStorage(index, df,"INSERT FOOD NAME", t)
#         print(f"{t}: ")
#         counter = 1
#         for i in all_tips:
#             print(f"\ttip {counter}:", i) # tool.correct(i)
#             counter += 1
#     print()

# print("=== ", df["Name"].iloc[144], " ===", f' ({436})') #401 #144
# for t in types:
#     all_tips = foodStorage(144, df,"INSERT FOOD NAME", t)
#     print(f"{t}: ")
#     counter = 1
#     for i in all_tips:
#         # print(f"\ttip {counter}:", tool.correct(i)) 
#         print(f"\ttip {counter}:", i) #tool.correct(i)
#         counter += 1
# print(df["DOP_" + 'Refrigerate' + "_tips"].iloc[144])
# Whole wheat flour can be stored in to Refrigerate for 2.0 Years


'\nDiscoveries...\n1. Using the foundKeywords, find semantics of tweet to utilize particular keywords.\n    - natural language processing (simpler)\n    - "Red Hot Chili Peppers"\n\nThings to fix...\n1. Refrigerator, Freezer (ending words)\n2. some cases produce nan, figure out why\n3. type casting all min and max values to int, not completly working (possible fix after above)\n4. Test all index for accuracy (is it really not tip to give?)\n'